In [1]:
import numpy as np
from tensorboard_evaluation import *

In [12]:
x = np.load("/run/media/luke/Data/uni/SS2023/DL Lab/Project/qualifying/DDPG_exp2/replay/data.npz")

In [13]:
x.__dict__

{'_files': ['state.npy',
  'action.npy',
  'next_state.npy',
  'reward.npy',
  'not_done.npy'],
 'files': ['state', 'action', 'next_state', 'reward', 'not_done'],
 'allow_pickle': False,
 'max_header_size': 10000,
 'pickle_kwargs': {'encoding': 'ASCII', 'fix_imports': True},
 'zip': <zipfile.ZipFile file=<_io.BufferedReader name='/run/media/luke/Data/uni/SS2023/DL Lab/Project/qualifying/DDPG_exp2/replay/data_4actions.npz'> mode='r'>,
 'f': <numpy.lib.npyio.BagObj at 0x7f84cc380760>,
 'fid': <_io.BufferedReader name='/run/media/luke/Data/uni/SS2023/DL Lab/Project/qualifying/DDPG_exp2/replay/data_4actions.npz'>}

In [14]:
state = x["state"]
next_state = x["next_state"]
reward = x["reward"]
not_done = x["not_done"]
action = x["action"]

In [15]:
action.shape

(105000, 4)

In [17]:
np.max(action[:,3])

2.9745588709802906

In [10]:
import numpy as np
import torch


class ReplayBuffer(object):
	def __init__(self, state_dim, action_dim, max_size=int(1e6)):
		self.max_size = max_size
		self.ptr = 105000
		self.size = 105000

		self.state = state
		self.action = action
		self.next_state = next_state
		self.reward = reward
		self.not_done = not_done

		self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


	def add(self, state, action, next_state, reward, done):
		self.state[self.ptr] = state
		self.action[self.ptr] = action
		self.next_state[self.ptr] = next_state
		self.reward[self.ptr] = reward
		self.not_done[self.ptr] = 1. - done

		self.ptr = (self.ptr + 1) % self.max_size
		self.size = min(self.size + 1, self.max_size)


	def sample(self, batch_size):
		ind = np.random.randint(0, self.size, size=batch_size)

		return (
			torch.FloatTensor(self.state[ind]).to(self.device),
			torch.FloatTensor(self.action[ind]).to(self.device),
			torch.FloatTensor(self.next_state[ind]).to(self.device),
			torch.FloatTensor(self.reward[ind]).to(self.device),
			torch.FloatTensor(self.not_done[ind]).to(self.device)
		)
	
	def save(self,filename):
		np.savez(filename,state = self.state[:self.size,::],action = self.action[:self.size,::],\
			next_state = self.next_state[:self.size,::],reward = self.reward[:self.size,::],not_done =self.not_done[:self.size,::])
	

In [25]:
from air_hockey_challenge.framework.air_hockey_challenge_wrapper import AirHockeyChallengeWrapper
from air_hockey_agent.agent_builder_ddpg_exp2_hit import build_agent
# from air_hockey_challenge.environments.planar.hit import AirHockeyHit
from tensorboard_evaluation import *

In [26]:
env = AirHockeyChallengeWrapper(env="7dof-hit", interpolation_order=3, debug=False)
# policy = build_agent(env.env_info)
policy = build_agent(env.env_info) 

In [9]:
replay_buffer = ReplayBuffer(23, 5)

In [831]:
from air_hockey_challenge.utils.kinematics import inverse_kinematics, jacobian
i=1120
state, done = env.reset(), False
env.render()

In [873]:
# state = replay_buffer.state[i,:]
action_ = replay_buffer.action[i,:]
# print(action)
action = np.zeros((2,7))
des_pos = [action_[0],action_[1],0.1645]
# print(des_pos)

_,x = inverse_kinematics(policy.robot_model, policy.robot_data,des_pos)
action[0,:] = x
des_v = action_[2:]
jac = jacobian(policy.robot_model, policy.robot_data,policy.get_joint_pos(state))
inv_jac = np.linalg.pinv(jac)
joint_vel = des_v@inv_jac.T[:3,:]
action[1,:] = joint_vel
next_state, reward, done, _ = env.step(action)
state = next_state
env.render()
print("base",env.base_env.get_ee()[1])
print("policy",(jac@policy.get_joint_vel(state))[:])
i+=1

base [0.41601167 0.94092308 0.00703506 0.16203352 0.03662689 0.52028456]
policy [-0.15598626  0.23548583 -0.03901886 -0.16438065  0.22543091 -0.04693529]


In [10]:
main_dir = "/run/media/luke/Data/uni/SS2023/DL Lab/Project/qualifying/DDPG_exp2"
tensorboard_dir=main_dir + "/tensorboard/"
tensorboard = Evaluation(tensorboard_dir, "train", ["critic_loss","actor_loss"])


In [11]:
for epoch in range(15000):
    critic_loss=np.nan
    actor_loss=np.nan
    critic_loss,actor_loss = policy.train(replay_buffer,batch_size= 512)
    tensorboard.write_episode_data(epoch, eval_dict={ "critic_loss" : critic_loss,\
                "actor_loss":actor_loss})

In [13]:
policy.save(main_dir + f"/models/DDPG-v0_air-hockey_0")